#### 한국어 코퍼스 전처리
- 데이터셋 : Korpora에서 로딩
- 형태소 분석기 설정
- 단어사전 생성

[1] 모듈 로딩 및 데이터 준비 <hr>

In [3]:
from Korpora import Korpora                      # Open Korean Dataset
from konlpy.tag import *                         # 형태소 분석기
import spacy                                     # 형태소 분석기

from torchtext.data.utils import get_tokenizer   # Pytorch 자연어처리 모듈
from torchtext.vocab import *                    # Pytorch 자연어처리 단어사전 관련 모듈

from torch.utils.data import Dataset, DataLoader # Pytorch Dataset관련 모듈

import pandas as pd
import numpy as np

In [4]:
## 데이터 로딩
nsmc = Korpora.load('nsmc')


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\KDP-2\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\KDP

In [5]:
nsmc.train[0], nsmc.test[0]

(LabeledSentence(text='아 더빙.. 진짜 짜증나네요 목소리', label=0),
 LabeledSentence(text='굳 ㅋ', label=1))

In [6]:
## Test 데이터셋을 DataFrame으로 로딩
nsmcDF = pd.DataFrame(nsmc.test)

nsmcDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


[2] 커스텀 데이터셋 정의 <hr>

In [7]:
### 단어 사전을 생성 시 활용
class TextDataset(Dataset):
    def __init__(self, feature, label):
        self.feature = feature
        self.label = label
        self.length = feature.shape[0]

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        return self.feature.iloc[index], self.label.iloc[index]

In [8]:
## NSMC의 데이터셋 인스턴스 생성
nsmcDS = TextDataset(nsmcDF['text'], nsmcDF['label'])


In [9]:
for f, l in nsmcDS:
    print(f, l)
    break

굳 ㅋ 1


[3] 텍스트 데이터 전처리 <hr>
- 토큰화 / 정제 (불용어, 구두점, 오타...)
- 단어사전 생성
- 문장 ==> 수치화

In [10]:
### 토커나이즈 생성
okt_tokenizer = get_tokenizer(Okt().morphs)
spacy_tokenizer = get_tokenizer('spacy', language = 'ko_core_news_lg')

In [11]:
### 텍스트 Dataset에서 처리할 데이터 셋만 로딩해서 토큰화 진행하는 반복자 함수
def generateToken(dataset):
    for text, label in dataset:
        yield okt_tokenizer(text) 

In [20]:
### 단어 사전 생성
nsmcVocab = build_vocab_from_iterator(iterator = generateToken(nsmcDS), 
                                      specials = ['<PAD>', '<UNK>'])
nsmcVocab.set_default_index(nsmcVocab['<UNK>'])

In [21]:
nsmcVocab.get_itos()

['<PAD>',
 '<UNK>',
 '.',
 '이',
 '영화',
 '의',
 '..',
 '가',
 '에',
 '...',
 '을',
 '도',
 '들',
 ',',
 '는',
 '를',
 '은',
 '너무',
 '?',
 '한',
 '다',
 '정말',
 '만',
 '진짜',
 '적',
 '!',
 '로',
 '점',
 '으로',
 '에서',
 '평점',
 '연기',
 '것',
 '과',
 '~',
 '최고',
 '내',
 '그',
 '나',
 '인',
 '잘',
 '와',
 '안',
 '생각',
 '게',
 '이런',
 '못',
 '왜',
 '스토리',
 '....',
 '이다',
 '드라마',
 '사람',
 '감동',
 '하는',
 '1',
 '보고',
 '때',
 '더',
 '하고',
 '고',
 '말',
 '아',
 '감독',
 '배우',
 'ㅋㅋ',
 '내용',
 '그냥',
 '거',
 '중',
 '까지',
 '재미',
 '보다',
 '본',
 '요',
 '!!',
 '없는',
 '좀',
 '뭐',
 '시간',
 '수',
 '지',
 '봤는데',
 '쓰레기',
 '사랑',
 '볼',
 '네',
 '작품',
 '다시',
 '하나',
 '없다',
 '10',
 '할',
 '이건',
 '마지막',
 '2',
 '저',
 '같은',
 '정도',
 '있는',
 'ㅠㅠ',
 'ㅋ',
 '좋은',
 '완전',
 '처음',
 '대',
 '장면',
 '주인공',
 '입니다',
 'ㅋㅋㅋ',
 '이렇게',
 '액션',
 '최악',
 '보는',
 '걸',
 '지금',
 '이야기',
 '하',
 '끝',
 '임',
 '개',
 '3',
 '참',
 '별로',
 "'",
 '없고',
 '연출',
 '돈',
 '서',
 '느낌',
 '듯',
 '봐도',
 '기',
 '라',
 '재밌게',
 'ㅡㅡ',
 '별',
 '역시',
 '인데',
 '명작',
 '난',
 '많이',
 '이해',
 '라고',
 '^^',
 '그리고',
 '면',
 '때문',
 '여자',
 '이영화'